# Clase 23: Benchmarking y Optimización de Código en Python

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Pablo Badilla**




## Motivación

El flujo de trabajo en ciencia de datos consta de **numerosas rutinas de carga, procesamiento y visualización**. Lo ideal es que diseñemos estas rutinas de la forma más optima posible con el fin de reducir recursos, tiempos de carga utilizados y sus costos asociados.


## Lenguajes de Programación

El lenguaje de máquina es el conjunto de instrucciones que el hardware es capaz de interpretar y procesar.
A través de estas instrucciones podemos lograr que nuestro procesador ejecute distintos tipos de acciones muy básicas. 
Este conjuntos de lenguajes es comunmente conocido como *lenguaje de bajo nivel*

<center>
<img src='./resources/codigo_maquina.png' width=400 />
<center/>

<center>Por suerte no tenemos que si quiera pensar en esto...</center>
    
<center> 
    Fuente: <a href='https://en.wikipedia.org/wiki/Machine_code#/media/File:W65C816S_Machine_Code_Monitor.jpeg'>Wikipedia </a>
</center>
    
    
    


La idea general de un lenguaje de programación que conocemos hasta ahora (como `python`) es tener una forma *humana* de comunicarse con la máquina. Es decir, que nos permitan escribir código de forma sencilla y que sean luego convertidos al lenguaje que la máquina sea capaz de entender.

Estos lenguajes se denominan *lenguajes de alto nivel*.

---

## Lenguajes Compilados vs Intepretados

Existen dos enfoques principales para convertir un código de lenguaje de alto nivel a uno de bajo nivel: que el lenguaje sea **Compilado** o **Interpretado**.

<center>
<img src='./resources/tipos_lenguajes.png' width=800/>

</center>

---

## Computación de alto Rendimiento con Python

Python es utilizado transversalmente, ya sea en la industria o en la academia. Dentro de sus cualidades se encuentra la portabilidad de código, sintaxis intuitiva, disponibilidad de herramientas y documentación. Sin embargo, al ser un lenguaje interpretado se pierden ciertas características intrínsicas de los lenguajes de bajo nivel como C, C++ y Fortran.

En esta y la próxima clase estudiaremos distintas herramientas para mejorar el rendimiento del interprete, como el uso eficiente de objetos base y la aplicación de técnicas de paralelismo y compilación utilizando tanto librerías nativas, como desarrolladas por terceros. 


> **Pregunta ❓:** ¿Será conveniente programar siempre pensando crear código óptimo?

---

## Optimización del Código

 Como directriz general, se recomienda llevar el proceso de desarrollo en dos etapas:
 
1. La primera consiste en **generar código correcto, comprensibles y mantenibles**, evitando la sobre-optimización prematura de código. 

2. Como segunda etapa, se recomienda comenzar con los procesos de **optimización de código**. Esto pues, las herramientas que permiten mejorar los aspectos computacionales, interfieren en la sencillez del código, entorpeciendo los procesos de depuración y mantención. 

Una vez que las rutinas están implementadas de manera correcta, la mejor manera de enfocar los esfuerzos, pasa por **perfilar** (*profiling*) el código. Esto consiste en encontrar las zonas de código criticas en cuanto a carga computacional. La manera más directa de encontrar estas zonas, es por medio del uso de contadores de tiempo o *timers*.



### Medición del Tiempo de Ejecución ⏰


El tiempo de ejecución es el tiempo tomado por algun segemento de código, función en completar su ejecución.

En Python, la forma más sencilla de medir el tiempo de ejecución es a través de la librería `time`. El ejemplo siguiente muestra como utilizarla.

In [1]:
import time
from math import cos, sin

Definimos un rango de datos a operar 

In [5]:
x = [0.1 * i for i in range(100000)]

x[0:10]  # veamos los datos

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9]

Luego definimos la función que mediremos. Esta simplemente calcula $(\sin(val) + \cos(val)^2)^{1/3}$ y luego retorna su valor.

In [6]:
def func_1(val):
    return (sin(val) + cos(val) ** 2) ** (1 / 9)

Ahora, estudiamos el tiempo de ejecución por medio de la función `process_time`.

In [19]:
# tiempo inicial
t0 = time.process_time()

for i in x:
    func_1(i)

# tiempo final
t1 = time.process_time()

# el tiempo transcurrido es simplemente el delta entre t1 y t0
print("Tiempo transcurrido", t1 - t0)

Tiempo transcurrido 0.04332430000000009


> **Pregunta ❓:** ¿Si ejecutamos nuevamente la celda anterior, obtendremos los mismos tiempos? ¿Existirá alguna forma más consistente de medir el tiempo de ejecución del código?

---

### `timeit`

En algunas ocasiones se desea medir el tiempo de ejecución para tareas sencillas, la librería estándar de Python provee el módulo `timeit`. En la práctica, una llamada de `timeit` ejecuta por defecto 10.000.000 el código (variable según cuánto se demore el proceso) y repite 7 veces el experimento. Lluego reporta el tiempo de ejecución promedio.

Este puede ser utilizado directamente en la consola interactiva IPython o en notebooks de Jupyter por medio del comando mágico `%timeit` para el caso de una linea de código y `%%timeit` para medir toda la celda. 

Documentación de `%timeit`: [Timeit Magic en la documentación de Ipython](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)



**Ejemplo**


Medimos la eficiencia de la implementación original de python de `cos`.

In [20]:
%timeit cos(0.5)

57.1 ns ± 5.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Y lo comparamos con el tiempo de ejecución promedio para la función coseno de `numpy`.

In [21]:
import numpy as np

In [22]:
%timeit np.cos(0.5)

872 ns ± 118 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [23]:
%timeit func_1(100)

375 ns ± 13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


> **Pregunta ❓:** ¿Se podrá medir el tiempo que toma cada instrucción por separado?

---

### Perfilador `line_profiler` y Benchmarking

Un **perfilador**  (*profiler*) es un programa que ejecuta una función y monitorea sus subfunciones, obteniendo métricas de rendimiento como el consumo de tiempo y memoria. 

Por otra parte, el ***benchmarking*** consiste en medir el rendimiento de ciertos segmentos de código rendimiento antes y después de aplicar técnicas de optimización. Es similar a establecer un baseline de un modelo y luego, optimizarlo y medir el incremento de rendimiento. 


IPython provee de un perfilador de código dado por la orden `%prun`. Sin embargo, es un poco complejo entender su funcionamiento. Por ende, utilizaremos `line_profiler` que entrega los resultados en un formato más sencillo.


In [24]:
!pip install line_profiler
%load_ext line_profiler


**Ejemplo**

1. Perfilaremos la siguiente función utilizando `%line_profiler`. En primera instancia se define tal función `benchmark_sum` que acumula para cada `i` en `n` los valores de la siguiente forma:

- Por cada $i$ en $n$:

    - `to_sum =` `[`${\frac{j}{2}}^n + (j - n)^{\frac{n}{3}}$ `for j in range(n)]`

    - `sum_ =` $\sum_{i=0}^{n}$ `to_sum`$_{i}$ 


In [25]:
def benchmark_sum(n):
    """Funcion de referencia que suma n elementos transformados"""
    ac = []

    for i in range(n):
        to_sum = [(j // 2) ** n + (j - n) ** (n // 3) for j in range(n)]
        sum_ = sum(to_sum)
        ac.append(sum_)

    return ac

Ahora, perfilamos la función con `%line_profiler`

In [26]:
%lprun -f benchmark_sum benchmark_sum(500)

Timer unit: 1e-06 s

Total time: 1.47807 s
File: /tmp/ipykernel_5653/1295444324.py
Function: benchmark_sum at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def benchmark_sum(n):
     2                                               """Funcion de referencia que suma n elementos transformados"""
     3         1         20.0     20.0      0.0      ac = []
     4                                           
     5       501        430.0      0.9      0.0      for i in range(n):
     6       500    1410466.0   2820.9     95.4          to_sum = [(j // 2) ** n + (j - n) ** (n // 3) for j in range(n)]
     7       500      65484.0    131.0      4.4          sum_ = sum(to_sum)
     8       500       1668.0      3.3      0.1          ac.append(sum_)
     9                                           
    10         1          0.0      0.0      0.0      return ac

EL resultado corresponde las mediciones temporales de cada función involucrada en la ejecución de `benchmark_sum(500)`. En este caso, la mayoria del tiempo se utiliza en la ejecución del list comprehension. 

Esto indica que la mejor manera de optimizar el código de `benchmark_sum`pasa por optimizar tal sección del código.

---

### Ejemplo Benchmark Numpy

Definimos en la siguiente celda una versión optimizada de la función anterior usando `numpy` y medimos.

In [27]:
def benchmark_sum_numpy(n):
    """Funcion de referencia que suma n elementos transformados"""
    ac = []

    for i in range(n):
        to_sum = np.arange(i, dtype="float128")
        to_sum = to_sum // 2.0 ** n + (to_sum - n) ** (n // 3.0)
        sum_ = np.sum(to_sum)
        ac.append(sum_)

    return ac

In [28]:
%lprun -f benchmark_sum_numpy benchmark_sum_numpy(500)

Timer unit: 1e-06 s

Total time: 0.130765 s
File: /tmp/ipykernel_5653/2288543561.py
Function: benchmark_sum_numpy at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def benchmark_sum_numpy(n):
     2                                               """Funcion de referencia que suma n elementos transformados"""
     3         1          2.0      2.0      0.0      ac = []
     4                                           
     5       501        317.0      0.6      0.2      for i in range(n):
     6       500      19385.0     38.8     14.8          to_sum = np.arange(i, dtype="float128")
     7       500      98077.0    196.2     75.0          to_sum = to_sum // 2.0 ** n + (to_sum - n) ** (n // 3.0)
     8       500      12143.0     24.3      9.3          sum_ = np.sum(to_sum)
     9       500        841.0      1.7      0.6          ac.append(sum_)
    10                                           
    11         1       

**Ejercicio**

1. Los perfiladores `%prun` y `%lprun` tienen en común los argumentos -D, -T y -r utilice el parámetro `?` para investigar estas opciones.

---

### `memory_profiler`

Es posible perfilar uso de memoria, para ello existen los comandos mágicos `%memit` y `%mprun`. Para utilizarlos es necesario instalar el módulo `memory_profiler` y cargarlo mediante

```python
%load_ext memory_profiler
```

**Ejemplo**

In [29]:
!pip install memory_profiler

In [30]:
%load_ext memory_profiler

En primera instancia perfilamos utilizando la linea mágica `%memit`, la cual es equivalente a `%timeit` pero ofrece medidas sobre el uso de memoria. 

In [31]:
%memit benchmark_sum(500)

peak memory: 80.57 MiB, increment: 0.57 MiB


In [32]:
%memit benchmark_sum_numpy(500)

peak memory: 80.58 MiB, increment: 0.00 MiB


Se puede observar un uso de memoria en torno a 90 MB.

De manera análoga a `%prun` la librería `memory_profiler` permite utilizar `%mprun`, con la cual se pueden obtener descripciones linea a linea del uso de memoria. El uso de este comando es un poco más restrictivo, pues solo permite medir funciones definidas en módulos (no dentro de un notebook). Para ello, se crea el módulo `memory_demo`. La manera sencilla de hacer esto, es mediante el comando mágico `%%file` este permite crear archivos en el directorio de trabajo actual, utilizando el código dentro de una celda de jupyter. 

Se procede a generar el módulo que contiene el código de `benchmark_sum` utilizando el comando `%%file`.




In [33]:
%%file bench_module.py


def benchmark_sum(n):
    """Funcion de referencia que suma n elementos transformados"""
    ac = []

    for i in range(n):
        to_sum = [(j // 2) ** n + (j - n) ** (n // 3) for j in range(n)]
        sum_ = sum(to_sum)
        ac.append(sum_)

    return ac

Overwriting bench_module.py


a continuación, se importa el módulo creado y se perfuila su memoria mediante:

In [34]:
from bench_module import benchmark_sum

%mprun -f benchmark_sum benchmark_sum(5000)

*** KeyboardInterrupt exception caught in code being profiled.


Filename: /home/pablo/Proyectos/MDS7202/MDS7202/clases/2022-01/23-Benchmark_de_codigo/bench_module.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     3     80.7 MiB     80.7 MiB           1   def benchmark_sum(n):
     4                                             """Funcion de referencia que suma n elementos transformados"""
     5     80.7 MiB      0.0 MiB           1       ac = []
     6                                         
     7    146.6 MiB      0.0 MiB          38       for i in range(n):
     8    146.6 MiB     65.8 MiB      187429           to_sum = [(j // 2) ** n + (j - n) ** (n // 3) for j in range(n)]
     9    146.6 MiB      0.0 MiB          37           sum_ = sum(to_sum)
    10    146.6 MiB      0.0 MiB          37           ac.append(sum_)
    11                                         
    12                                             return ac

El resultado se muestra en pantalla, obteniendo detalles linea a linea.

---

## Optimización del Código Nativo

Una de las manera más eficientes de mejorar el rendimiento de aplicaciones es por medio del uso de algoritmos más eficientes en conjunción de estructuras de datos mejor diseñadas. A continuación, se estudian los algoritmos y estructuras de datos presentes de manera nativa en Python que permiten acelerar ciertas rutinas. 

En términos generales, los algoritmos pueden ser clasificados según su **complejidad computacional**. Esta clasificación se expresa según la notación de **O-grande, que corresponde a una cota superior de las operaciones requeridas para ejecutar una tarea.**

Ejemplos: 

- $O(n)$ requiere ejecutar una operación por cada elemento de un arreglo.
- $O(1)$ requiero para acceder a cierta llave de un diccionario. Noten que esto es totalmente independiente del número de datos que tenga el diccionario.

**Ejemplo**

Se genera un lista, a cada uno de sus elementos se le realiza una operación básica.

In [35]:
lista = list(range(10))
lista

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [36]:
for i in range(len(lista)):
    lista[i] += 100

In [37]:
lista

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]

In [38]:
lista = np.arange(0, 10)
lista

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [39]:
lista + 100

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109])

En este algoritmo, la operación `lista[i]+=100` es repetida tantas veces como elementos hay en `lista`, que corresponde al tamaño de los datos de entrada. Al observar la que las operaciones realizadas por este algoritmo son proporcionales a la cantidad de elementos de `lista`, se puede decir que su tiempo de ejecución es $O(N)$ donde `N = len(lista)`. 

### Optimización de Operaciones con Listas

Las listas de Python son **colecciones ordenadas de elementos**, estás se encuentran clasificadas como *arreglos*, que a la vez corresponden a una estructura de datos caracterizada por contener elementos contiguos en bloques de memoria, cada uno de los cuales contienen una referencia a un objeto de Python. La ventaja de las listas recae en la facilidad que entregan par acceder, modificar y agregar elementos. 

#### Acceder y Modificar

Dado que acceder y modificar elementos de una lista corresponde a acceder a espacios de memoria que a priori no dependen de la longitud de la lista, se dice que estas operaciones tienen complejidad $O(1)$. 


```python
[0, 1, 2, 3]
#   ↑
# acceder y modificar cualquier elemento es tiempo constante O(1)    
```

#### Agregar al Final

Por otra parte, para agregar un elemento a una lista por medio de `.append()`, puede requerirse re-ubicar la memoria del arreglo asociado, operación que toma un tiempo de $O(N)$. Sin embargo, tal operación es muy poco frecuente, pues por lo general se tiene acceso a bloques de memoria contiguos, por tal motivo, se dice que la operación `.append()` tiene un tiempo esperado de ejecución de $O(1)$. 

```python
[0, 1, 2, 3]

#si ejemcutamos append(4)

[0, 1, 2, 3, 4]
#            ↑
# agregar por lo general es O(1)    
```


#### Agregar (`insert`) /Eliminar al inicio

Para agregar o eliminar datos al inicio de un arreglo, se requiere hacer una traslación (o *shift*) de los demás elementos por lo que tal operación toma un tiempo de $O(N)$. Para agregar o remover elementos de un arreglo en una posición distinta a la última, se opera de manera análoga. 

```python
[0, 1, 2, 3]

#si ejemcutamos insert(0, -1) tendremos que desplazar todo el arreglo a la derecha.

[-1, 0, 1, 2, 3]
# ↑, →, →, →, →
# agregar al inicio implicará desplazar todo. Es decir, O(n)
```


```python
[0, 1, 2, 3]

#si ejemcutamos pop(0) tendremos que desplazar todo el arreglo a la izquierda.

[ 1, 2, 3]
# ←, ←, ←
# eliminar al inicio implicará desplazar todo. Es decir, O(n)
```



**Ejemplo**

Se definen listas para estudiar la complejidad de ciertos métodos empíricamente. Se definen los parámetros

In [40]:
n_0, n_1, n_2 = (int(10e5), int(5 * 10e5), int(10e6))

In [41]:
n_0

1000000

In [42]:
n_1

5000000

In [43]:
n_2

10000000

Se generan una funciones de referencia

In [46]:
def copy_objs(obj_0, obj_1, obj_2):
    """Abstraccion auxiliar para copiar elementos."""
    return (obj_0.copy(), obj_1.copy(), obj_2.copy())


def bench_pop(l_0, l_1, l_2, arg=-1):
    """Funcion de referncia para eliminacion de elementos."""

    l_0.pop(arg)
    l_1.pop(arg)
    l_2.pop(arg)


def bench_append(l_0, l_1, l_2, arg=1):
    """Funcion de referencia para insertar 1 con append."""

    l_0.append(arg)
    l_1.append(arg)
    l_2.append(arg)


def bench_insert(l_0, l_1, l_2, args=(0, 1)):
    """Funcion de referncia para insertar 1 con insert."""

    l_0.insert(*args)
    l_1.insert(*args)
    l_2.insert(*args)

Se construye el test 

In [47]:
lista_0, lista_1, lista_2 = (list(range(n_0)), list(range(n_1)), list(range(n_2)))

In [48]:
len(lista_0)

1000000

In [49]:
len(lista_1)

5000000

In [50]:
len(lista_2)

10000000

#### Eliminar el ultimo elemento

In [51]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante
%lprun -f bench_pop bench_pop(l_0,l_1,l_2)

Timer unit: 1e-06 s

Total time: 1.2e-05 s
File: /tmp/ipykernel_5653/3241826286.py
Function: bench_pop at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def bench_pop(l_0, l_1, l_2, arg=-1):
     7                                               """Funcion de referncia para eliminacion de elementos."""
     8                                           
     9         1          9.0      9.0     75.0      l_0.pop(arg)
    10         1          2.0      2.0     16.7      l_1.pop(arg)
    11         1          1.0      1.0      8.3      l_2.pop(arg)

#### Eliminar el primer elemento

In [52]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_pop bench_pop(l_0,l_1,l_2,0)

Timer unit: 1e-06 s

Total time: 0.020539 s
File: /tmp/ipykernel_5653/3241826286.py
Function: bench_pop at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def bench_pop(l_0, l_1, l_2, arg=-1):
     7                                               """Funcion de referncia para eliminacion de elementos."""
     8                                           
     9         1       1208.0   1208.0      5.9      l_0.pop(arg)
    10         1       7373.0   7373.0     35.9      l_1.pop(arg)
    11         1      11958.0  11958.0     58.2      l_2.pop(arg)

#### Insertar 1 en la ultima posicion

In [54]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante (casi seguramente)
%lprun -f  bench_append bench_append(l_0,l_1,l_2)

Timer unit: 1e-06 s

Total time: 0.000847 s
File: /tmp/ipykernel_5653/3241826286.py
Function: bench_append at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def bench_append(l_0, l_1, l_2, arg=1):
    15                                               """Funcion de referencia para insertar 1 con append."""
    16                                           
    17         1        339.0    339.0     40.0      l_0.append(arg)
    18         1          7.0      7.0      0.8      l_1.append(arg)
    19         1        501.0    501.0     59.1      l_2.append(arg)

#### Insertar 1 en la primera posicion

In [55]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_insert bench_insert(l_0,l_1,l_2)

Timer unit: 1e-06 s

Total time: 0.028344 s
File: /tmp/ipykernel_5653/3241826286.py
Function: bench_insert at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def bench_insert(l_0, l_1, l_2, args=(0, 1)):
    23                                               """Funcion de referncia para insertar 1 con insert."""
    24                                           
    25         1       1348.0   1348.0      4.8      l_0.insert(*args)
    26         1       6065.0   6065.0     21.4      l_1.insert(*args)
    27         1      20931.0  20931.0     73.8      l_2.insert(*args)

---

### Double-Ended Queue: `deque` 


<img src='./resources/deque.png'/>

<center>Fuente: <a href='https://medium.com/@rasmussen.matias/fun-with-deques-in-python-31942bcb6321'>Matias Rasmussen en Medium </a>
    
</center>

Para efectuar inserciones de manera eficiente (siempre en tiempo constante) Se puede utilizar la estructura de datos `deque` del módulo `collections`. Estas estructuras se comportan como listas, están diseñadas para acelerar la inserción de objetos y añaden los métodos `.popleft` y `.appendleft`. 




**Ejemplo**

Se compara `.popleft` en *deques* con `.pop(0)` en listas.

In [57]:
from collections import deque


def bench_pop_left(d_0, d_1, d_2):
    """Funcion de referncia para eliminacion de elementos."""

    d_0.popleft()
    d_1.popleft()
    d_2.popleft()


def bench_append_left(d_0, d_1, d_2, val=1):
    """Funcion de referncia para insertar 1 con insert."""

    d_0.appendleft(val)
    d_1.appendleft(val)
    d_2.appendleft(val)

Se definen los objetos sobre los que se trabajará

In [58]:
deque_0, deque_1, deque_2 = tuple(map(deque, [lista_1, lista_1, lista_2]))

In [59]:
d_0, d_1, d_2 = copy_objs(deque_0, deque_1, deque_2)

# Se observa un tiempo constante
%lprun -f bench_pop_left bench_pop_left(d_0, d_1, d_2)

Timer unit: 1e-06 s

Total time: 7e-06 s
File: /tmp/ipykernel_5653/1103117999.py
Function: bench_pop_left at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def bench_pop_left(d_0, d_1, d_2):
     5                                               """Funcion de referncia para eliminacion de elementos."""
     6                                           
     7         1          3.0      3.0     42.9      d_0.popleft()
     8         1          2.0      2.0     28.6      d_1.popleft()
     9         1          2.0      2.0     28.6      d_2.popleft()

In [60]:
%lprun -f bench_append_left bench_append_left(d_0, d_1, d_2)

Timer unit: 1e-06 s

Total time: 6e-06 s
File: /tmp/ipykernel_5653/1103117999.py
Function: bench_append_left at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def bench_append_left(d_0, d_1, d_2, val=1):
    13                                               """Funcion de referncia para insertar 1 con insert."""
    14                                           
    15         1          4.0      4.0     66.7      d_0.appendleft(val)
    16         1          1.0      1.0     16.7      d_1.appendleft(val)
    17         1          1.0      1.0     16.7      d_2.appendleft(val)

---

### Optimización de Operaciones con Diccionarios

La gran flexibilidad de los diccionarios los hacen un objeto central en el uso de Python. Estos son implementaciones de *hash maps*, es decir, son estructuras de datos construidas por medio de asociaciones *llave - valor*, donde a cada llave, se asigna un índice especifico, de tal manera que el valor de tal índice puede ser ordenado en un arreglo. Por tal motivo, los diccionarios son altamente eficientes en procesos de eliminación, acceso e inserción teniendo un tiempo promedio de ejecución de $O(1)$. 


<div align='center'>
    <img src='./resources/hashmap.png' width=500 />
    
    

              
</div>

<div align='center'>
    Fuente: <a href='https://techmastertutorial.in/java-collection-internal-hashmap.html'>Java HashMap en techmastertutorial.in</a>
    </div>
<br> 

Para acceder a los índices dados por el *hash map* se puede utilizar la función `hash` de Python, esta opera sobre distintos tipos de datos.




**Ejemplo**

Se aplica `hash` a diferentes objetos

In [61]:
print("hash string: ", hash("MDS7202"))
print("hash int:", hash(1234))
print("hash tuple", hash(("a", "b", "c")))

hash string:  -1629309444246636649
hash int: 1234
hash tuple -7665359673442551369


Un objeto puede ser operado por `hash` (*hashable object*) si tiene un método `__hash__` y puede ser comparado por medio de `__eq__` por ejemplo. Si un objeto es *hashable* significa que puede ser utilizado como llave de un diccionario, en general, todos los objetos inmutables de Python son *hashables* mientras que las listas y diccionarios, por ser inmutables, no lo son.  



#### `defaultdict`

In [62]:
from collections import defaultdict

Los objetos `defaultdict` poseen todas las funcionalidades de un diccionario pero añaden el método `.__missing__()` con el cual se proveen valores por defecto, los cuales se asignan a una nueva llave, es decir, permiten inicializar diccionarios entregando solo el valor de la llave (y no su valor asociado), pues a cada llave nueva, se asigna un valor por defecto de manera automática. 


In [63]:
# list es el valor por defecto que tendrá cada llave del diccionario,
# aunque esta no se haya definido.
d = defaultdict(list)
d

defaultdict(list, {})

In [64]:
"hola" in d

False

In [65]:
# esto inicializa la variable hola.
d["hola"]

[]

In [66]:
d

defaultdict(list, {'hola': []})

In [67]:
d["hola2"]

[]

In [68]:
d

defaultdict(list, {'hola': [], 'hola2': []})

In [69]:
"hola" in d

True

El comportamiento anterior, como vimos anteriormente, no es posible en un diccionario común y corriente:

In [70]:
d2 = {}
d2["hola"]

KeyError: 'hola'

In [71]:
to_group = [
    ("a", 1),
    ("b", 2),
    ("c", 3),
    ("b", 4),
    ("d", 1),
]

Esto es lo que quisieramos lograr:

```python
salida_esperada = [('a', [1]), 
                   ('b', [2, 4]), 
                   ('c', [3]), 
                   ('d', [1])]
```


In [72]:
D = defaultdict(list)

for k, v in to_group:
    # versión sin defaultdict
    #if k in D:
    #    D[k].append(v)
    #else: 
    #    D[k] = [v]
        
    D[k].append(v)

D

defaultdict(list, {'a': [1], 'b': [2, 4], 'c': [3], 'd': [1]})

#### Ejemplo Eficiencia Diccionarios vs Listas

El siguiente ejemplo muestra el como optimizar los diccionarios es que permiten buscar palabras de manera rápida en una *lista de documentos*. 

Por ejemplo, *El hombre imaginario*. (aquí asumimos que cada elemento del arreglo es un documento por separado).

In [73]:
with open("./resources/text.txt", "r") as file:
    lines = file.readlines()
    lines = [l.rstrip(" \n") for l in lines]

lines

['El hombre imaginario',
 'vive en una mansión imaginaria',
 'rodeada de árboles imaginarios',
 'a la orilla de un río imaginario',
 '',
 'De los muros que son imaginarios',
 'penden antiguos cuadros imaginarios',
 'irreparables grietas imaginarias',
 'que representan hechos imaginarios',
 'ocurridos en mundos imaginarios',
 'en lugares y tiempos imaginarios',
 '',
 'Todas las tardes tardes imaginarias',
 'sube las escaleras imaginarias',
 'y se asoma al balcón imaginario',
 'a mirar el paisaje imaginario',
 'que consiste en un valle imaginario',
 'circundado de cerros imaginarios',
 '',
 'Sombras imaginarias',
 'vienen por el camino imaginario',
 'entonando canciones imaginarias',
 'a la muerte del sol imaginario',
 '',
 'Y en las noches de luna imaginaria',
 'sueña con la mujer imaginaria',
 'que le brindó su amor imaginario',
 'vuelve a sentir ese mismo dolor',
 'ese mismo placer imaginario',
 'y vuelve a palpitar',
 'el corazón del hombre imaginario']

Supongamos que queremos buscar la palabra `'imaginario'` en cada documento. Es posible hacer esto recorriendo todo cada vez que ejecutamos la búsqueda según el siguiente código.

In [74]:
to_search = "imaginario"

found = [line for line in lines if to_search in line]
found

['El hombre imaginario',
 'rodeada de árboles imaginarios',
 'a la orilla de un río imaginario',
 'De los muros que son imaginarios',
 'penden antiguos cuadros imaginarios',
 'que representan hechos imaginarios',
 'ocurridos en mundos imaginarios',
 'en lugares y tiempos imaginarios',
 'y se asoma al balcón imaginario',
 'a mirar el paisaje imaginario',
 'que consiste en un valle imaginario',
 'circundado de cerros imaginarios',
 'vienen por el camino imaginario',
 'a la muerte del sol imaginario',
 'que le brindó su amor imaginario',
 'ese mismo placer imaginario',
 'el corazón del hombre imaginario']

In [75]:
%timeit found = [line for line in lines if to_search in line]

1.88 µs ± 74.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Se debe considerar que el tiempo de ejecución asociado a consultar por una palabra es $O(N)$. Para mejorar esto, se puede construir un diccionario, donde a cada palabra se asocie un índice, donde este último corresponde al la linea (o documento si se prefiere) al que pertenece. Esto se puede hacer mediante el siguiente código

In [76]:
index = defaultdict(list)

for i, line in enumerate(lines):

    for word in line.split():
        index[word].append(i)

In [77]:
index

defaultdict(list,
            {'El': [0],
             'hombre': [0, 30],
             'imaginario': [0, 3, 14, 15, 16, 20, 22, 26, 28, 30],
             'vive': [1],
             'en': [1, 9, 10, 16, 24],
             'una': [1],
             'mansión': [1],
             'imaginaria': [1, 24, 25],
             'rodeada': [2],
             'de': [2, 3, 17, 24],
             'árboles': [2],
             'imaginarios': [2, 5, 6, 8, 9, 10, 17],
             'a': [3, 15, 22, 27, 29],
             'la': [3, 22, 25],
             'orilla': [3],
             'un': [3, 16],
             'río': [3],
             'De': [5],
             'los': [5],
             'muros': [5],
             'que': [5, 8, 16, 26],
             'son': [5],
             'penden': [6],
             'antiguos': [6],
             'cuadros': [6],
             'irreparables': [7],
             'grietas': [7],
             'imaginarias': [7, 12, 13, 19, 21],
             'representan': [8],
             'hechos': [8],
     

En el diccionario generado, hacer búsquedas es de orden $O(1)$, luego para la misma consulta antes hecha se tiene

In [78]:
%%timeit
res = index[to_search]
[lines[i] for i in res]

1.01 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Es decir, un aumento substancial de rendimiento. Cabe mencionar, que este procedimiento solo tiene sentido si se busca hacer una cantidad alta de consultas sobre un arreglo de lineas/documentos, esto pues, el tiempo de preprocesamiento para generar el indexado por diccionario puede ser muy alto.

---

### Optimización con uso de Memoización

También se puede mejorar el rendimiento de aplicaciones por medio de un uso eficiente de la memoria, una de las ideas tras esta premisa es la de guardar los resultados de operaciones intensivas en un espacio de memoria llamado *cache*, este espacio puede estar ubicado en memoria (RAM), disco o almacenada de manera remota. El acto de guardar resultados en memoria para luego utilizarlos de manera directa se denomina **memoización**. y es una forma de *chaching* o uso de memorias *cache*.

Python ofrece el decorador `@lru_cache` accesible desde la librería base `functools`. Este decorador puede ser utilizado de manera sencilla para guardar resultados en memoria y luego accederlos. 

**Ejemplo**

Se utiliza el decorador `@lru_cache` sobre una función sencilla. En primera instancia se importa el módulo necesario.

In [80]:
from functools import lru_cache

Se define la función a memoizar y se aplica el decorador

In [81]:
@lru_cache
def simple_func(x, y):
    """Funcion de prueba para memoizar."""

    print("Obteniendo el resultado...")

    return x**y + y**x

Para comprobar el funcionamiento del decorador se llama la función dos veces sobre el mismo argumento.

In [82]:
args = (2, 5)
simple_func(*args)

Obteniendo el resultado...


57

Se repite el procedimiento 

In [83]:
simple_func(*args)

57

En este último caso se observa que el resultado es obtenido directamente desde la memoria. 

**Ejercicios**

`@lru_cache` es un decorador que acepta argumentos de entrada, permitiendo el uso del `max_size`. Con este parámetro se especifica el tamaño máximo de memoria para el cache asociado a la función. 

1. Decore la función anterior indicando como parámetro `max_size = 8`.

2. ¿Qué ocurre cuando se llena el tamaño maximo y se realizan más cálculos? *Hint*: lru significa *least recently used*.

3. Acceda a la información del *cache* por medio del método `.cache_info()` de la función decorada. ¿Qué significa *hit* y *miss* en este contexto?


Un ejemplo más avanzado es el de memoizar funciones recursivas

**Ejemplo**

Se trabaja con la función recursiva `factorial` almacenando sus resultados en *cache*.


```python

(factorial(5))
(5 * factorial(4))
(5 * (4 *factorial(3)))
(5 * (4 * (3 *factorial(2))))
(5 * (4 * (3 * (2 * factorial(1)))))
(5 * (4 * (3 * (2 * 1))))
(5 * (4 * (3 * 2)))
(5 * (4 * 6))
(5 * 24)
(120)

```

In [84]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n - 1)

Se mide el tiempo de ejecución para `n=1000`.

In [85]:
%%timeit
factorial(1000)

574 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Se memoiza la función y se repite el experimento

In [86]:
@lru_cache(maxsize=1000)
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n - 1)

In [87]:
%%timeit
factorial(1000)

96.7 ns ± 5.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Con lo que se comprueba la eficiencia del método. 

> **Nota**: La ventaja de utilizar técnicas de *caching* tienen un costo, este radica en aumentar el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente. Antes de usar este tipo de estrategias, se recomienda estudiar la factibilidad, teniendo en cuenta las politicas de almacenamiento y acceso de los resultados y su relación con el rendimiento del programa que se desea implementar.